In [1]:
from pathlib import Path
import sys
a_path = str(Path('..').resolve())
sys.path.insert(0, a_path)
from bella.parsers import semeval_14
from bella.preprocessing import tokeniser
from bella.tokenisers import moses
from bella.word_embeddings import GloveCommonEmbedding
from bella.dataloaders import TargetSequence, LeftRightTargetSequence

Using TensorFlow backend.


In [2]:
rest_sem_dir = Path('..', '..', 'aspect datasets', 'semeval_2014')
rest_train = semeval_14(Path(rest_sem_dir, 'restaurants_train.xml'))
rest_test = semeval_14(Path(rest_sem_dir, 'restaurants_test.xml'))

rest_train_fp, rest_dev_fp = rest_train.to_json_file(['restaurants train', 'restaurants dev'], 
                                                     0.2, cache=False, random_state=42)
rest_test_fp = rest_test.to_json_file('restaurants test', cache=False)

In [3]:
tok = tokeniser(rest_train_fp, rest_dev_fp, rest_test_fp, 
                tokeniser_function=moses, 
                lower=True, filters='', oov_token='<UNK>')
train_data = TargetSequence(rest_train_fp, 32, tok, moses)
val_data = TargetSequence(rest_dev_fp, 32, tok, moses)

glove_rest = GloveCommonEmbedding(840, tok.word_index)
num_words, embedding_dim = glove_rest.embedding.shape

In [4]:
train_target_data = LeftRightTargetSequence(rest_train_fp, 32, tok, moses)
val_target_data = LeftRightTargetSequence(rest_dev_fp, 32, tok, moses)

In [102]:
from keras.layers import Layer
import keras.backend as K
import tensorflow as tf
class Average(Layer):
    def __init__(self, mask_zero=True, **kwargs):
        self.mask_zero = mask_zero
        self.supports_masking = True
        super(Average, self).__init__(**kwargs)

    def call(self, x, mask=None):
        x = K.print_tensor(x, message="Value of mask")
        tf.Print(x, [x])
        print(x)
        mask = K.print_tensor(mask, message="Value of mask")
        print(mask)
        if self.mask_zero:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask)
            x = x * mask
            return K.sum(x, axis=1) / (K.sum(mask, axis=1) + K.epsilon())
        else:
            return K.mean(x, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])
    
    def compute_mask(self, x, mask):
        return None

In [103]:
from keras.layers import Input, Embedding, LSTM, Bidirectional, concatenate, Dense, TimeDistributed
from keras.models import Model
target_text_input = Input(batch_shape=(None, None), dtype='int32',
                              name='target_text')
target_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim,
                             mask_zero=True, weights=[glove_rest.embedding], 
                             trainable=False, name='target_embedding')(target_text_input)
average = Average()(target_embedding)
out = Dense(3, activation='softmax')(average)



model = Model(inputs=target_text_input, outputs=out)
model.compile('adam', loss='categorical_crossentropy')

Tensor("average_25/Print:0", shape=(?, ?, 300), dtype=float32)
Tensor("average_25/Print_2:0", shape=(?, ?), dtype=bool)


In [104]:
target_input = train_data[0][0][1]
fun = K.function([model.layers[0].input],[model.layers[2].output])

In [79]:
train_data[0][1][0]

array([0, 9], dtype=int32)

In [105]:
model.fit(x=target_input[0:1], y=train_data[0][1][0:1])

Epoch 1/1
1/1 [==============================] - 0s 79ms/step - loss: 1.3313


In [100]:
fun([target_input[-1:]])

[array([[  1.69799998e-02,  -2.80155018e-02,  -3.17294970e-02,
          -1.85609013e-01,  -1.89264998e-01,   2.23354995e-01,
          -3.69859993e-01,   1.31480008e-01,   1.11800998e-01,
           1.18706501e+00,  -5.07764995e-01,   1.48595003e-02,
          -6.45980000e-01,  -2.37771988e-01,   4.52085018e-01,
          -2.33445004e-01,  -5.35239995e-01,   7.84284949e-01,
           1.98040009e-01,  -1.86841503e-01,  -3.29239994e-01,
          -9.70500037e-02,  -1.54610008e-01,   2.87600234e-03,
           9.70600024e-02,  -6.95900023e-01,  -5.02749998e-03,
          -1.33509994e-01,  -1.48579508e-01,  -8.21910024e-01,
           2.16110006e-01,   1.95395008e-01,  -1.62875995e-01,
          -4.64625001e-01,  -2.96939999e-01,   3.75808030e-01,
           3.47629994e-01,   1.38065502e-01,  -2.68836021e-01,
           1.19214997e-01,  -5.51974997e-02,   4.66300100e-02,
           2.93199942e-02,   4.57815006e-02,   5.08894995e-02,
           4.13325012e-01,  -1.58317998e-01,   5.652249

In [5]:
from keras.layers import Input, Embedding, LSTM, Bidirectional, concatenate, Dense, TimeDistributed
from keras.models import Model
def tclstm_method():
    target_text_input = Input(batch_shape=(None, None), dtype='int32',
                              name='target_text')
    target_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim,
                                 mask_zero=True, weights=[glove_rest.embedding], 
                                 trainable=True, name='target_embedding')
    left_text_input = Input(batch_shape=(None, None), dtype='int32',
                             name='left_text')
    
    sentence_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim,
                                   mask_zero=True, weights=[glove_rest.embedding], 
                                   trainable=True, name='sentence_embedding')

    model = Model(inputs=[left_text_input, target_text_input], outputs=softmax)
    return model

([array([[   0,    0,    0,    0,    9],
         [   0,    0,    0, 3607,   85],
         [   0,    0,    0,  282,   16],
         [   0,    0,    0, 2289,   67],
         [   0,    0,    0,   22,  248],
         [   0,    0,    0,  204,   67],
         [   0,    0,    0, 1035,   16],
         [   0,    0,    0, 3919,   67],
         [   0,    0,    0,   22,    9],
         [   0,    0,    0,   22,    9],
         [   0,    0,    0,    0, 4143],
         [   0,    0,    0,  614, 4171],
         [   0,    0,    0,    0,  207],
         [   0,    0,    5,  162,   38],
         [   0,   22,    2,  282,   16],
         [   0,    0,    0,    0,   16],
         [   0,    0,  106, 1057,  127],
         [   0,    0,  111,    1, 2375],
         [   0,    0,   78,  720,  248],
         [   0,    0,  177,    1,    9],
         [   0,    0,    0,    0,    9],
         [   0,    0,  223, 3133,    9],
         [   0,    0,   91,   19,   16],
         [   0,    0,    0,   19,   85],
         [   0, 

In [6]:
from keras.layers import Input, Embedding, LSTM, Bidirectional, concatenate, Dense
from keras.models import Model
text_sequences = Input(batch_shape=(None, None), dtype='int32',
                        name='Text_Input')
text_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim,
                           mask_zero=True, weights=[glove_rest.embedding], 
                           trainable=True, name='Text_Embedding')(text_sequences)
text_lstm = Bidirectional(LSTM(300, name='Text_LSTM'))(text_embedding)
target_sequences = Input(batch_shape=(None, None), dtype='int32',
                         name='Target_Input')
target_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim,
                             mask_zero=True, weights=[glove_rest.embedding], 
                             trainable=True, name='Target_Embedding')(target_sequences)
target_lstm = Bidirectional(LSTM(300, name='Target_LSTM'))(target_embedding)
combo = concatenate([text_lstm, target_lstm], name='Text_Target_Merge')
softmax = Dense(3, activation='softmax', name='Softmax')(combo)

model = Model(inputs=[text_sequences, target_sequences], outputs=softmax)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
import keras.backend as K
text_input = train_data[0][0][0]
target_input = train_data[0][0][1]
fun = K.function([model.layers[0].input],[model.layers[2].output])

In [7]:
model.fit_generator(train_data, validation_data=val_data, use_multiprocessing=True, workers=2)

Epoch 1/1
91/91 [==============================] - 32s 356ms/step - loss: 0.7914 - acc: 0.6522 - val_loss: 0.6667 - val_acc: 0.7295


In [24]:
model.fit_generator(train_data, validation_data=val_data, use_multiprocessing=True, workers=3)

Epoch 1/1
91/91 [==============================] - 30s 325ms/step - loss: 0.5709 - acc: 0.7678 - val_loss: 0.6401 - val_acc: 0.7351


In [28]:
texts = []
targets = []
labels = []
for i in range(len(train_data)):
    texts.extend(train_data[i][0][0].tolist())
    targets.extend(train_data[i][0][1].tolist())
    labels.extend(train_data[i][1].tolist())

In [32]:
from keras.preprocessing.sequence import pad_sequences
texts = pad_sequences(texts)
targets = pad_sequences(targets)

In [37]:
import numpy as np
np.array(labels)

array([[ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.]])

In [38]:
model.fit(x=[texts, targets], y=np.array(labels))

Epoch 1/1
2881/2881 [==============================] - 70s 24ms/step - loss: 0.4315 - acc: 0.8285


In [41]:
from bella.contexts import context
con = {"spans": [[170, 175]], "target": "plate", "text": "They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate when you touched it.", "sentiment": 0, "sentence_id": "restaurants_train296", "target_id": "restaurants_train2965"}
context(con, 'left', True)

['They did not have mayonnaise, forgot our toast, left out ingredients (ie cheese in an omelet), below hot temperatures and the bacon was so over cooked it crumbled on the plate']

In [98]:
text_input = data[0][0]
target_input = data[0][1]


TypeError: __call__() takes 2 positional arguments but 3 were given

In [20]:
[layer.name for layer in model.layers]

['Text_Input',
 'Target_Input',
 'Text_Embedding',
 'Target_Embedding',
 'bidirectional_3',
 'bidirectional_4',
 'Text_Target_Merge',
 'Softmax']

In [90]:
fun = K.function([model.layers[0].input, model.layers[1].input],[model.layers[7].output])

In [73]:
fun([*[data[0][0], data[0][1]]])

InvalidArgumentError: You must feed a value for placeholder tensor 'Target_Input_4' with dtype int32 and shape [32,?]
	 [[Node: Target_Input_4 = Placeholder[dtype=DT_INT32, shape=[32,?], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Target_Input_4', defined at:
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/andrew/Envs/Bella/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/andrew/Envs/Bella/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/andrew/Envs/Bella/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/andrew/Envs/Bella/lib/python3.6/site-packages/tornado/platform/asyncio.py", line 132, in start
    sel

In [72]:
fun([{'Text_Input': data[0][0], 'Target_Input': data[0][1]}])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'dict'

In [48]:
fun([data[0][0]])[0]

array([[[  4.42023985e-02,  -2.87089404e-02,   1.30871320e-02, ...,
          -4.66016456e-02,   1.05595612e-03,  -5.87926060e-03],
        [  4.42023985e-02,  -2.87089404e-02,   1.30871320e-02, ...,
          -4.66016456e-02,   1.05595612e-03,  -5.87926060e-03],
        [  4.42023985e-02,  -2.87089404e-02,   1.30871320e-02, ...,
          -4.66016456e-02,   1.05595612e-03,  -5.87926060e-03],
        ..., 
        [ -9.38460007e-02,   5.82960010e-01,  -1.92709994e-02, ...,
          -9.88039970e-02,   2.75379997e-02,   3.00419986e-01],
        [ -2.19049994e-02,  -1.89789996e-01,   3.55529994e-01, ...,
           2.54469998e-02,  -2.61300001e-02,  -9.11479965e-02],
        [  1.20010003e-02,   2.07509995e-01,  -1.25780001e-01, ...,
           1.38710007e-01,  -3.60489994e-01,  -3.50000001e-02]],

       [[  4.42023985e-02,  -2.87089404e-02,   1.30871320e-02, ...,
          -4.66016456e-02,   1.05595612e-03,  -5.87926060e-03],
        [  4.42023985e-02,  -2.87089404e-02,   1.30871320e-0

In [39]:
data = TargetSequence(rest_train_fp, 32, tok, moses)
from keras import backend as K

In [27]:
data[0][0]

array([[   0,    0,    0,    0,   19,   60,    3],
       [   0,    0,    0,    0, 3701,   56,   36],
       [   0,    0,    0,    0,  289,   16,    3],
       [   0,    0,    0,    0, 3841,   67,    3],
       [   0,    0,    0,    0,   22,    9,    3],
       [   0,    0,    0,    0,  619, 3952,    3],
       [   0,    0,    0,    0, 1416,  169,    3],
       [   0,    0,    0,    0, 1072,   16,    3],
       [   0,    0,    0,    0,   22,  247,    3],
       [   0,    0,    0,    0, 2626,   67,    3],
       [   0,    0,    0,    0,   22,    9,    3],
       [   0,    0,    0,    0, 4118,   85,   36],
       [   0,    0,    0,   16,    8,  370,    3],
       [   0,    0,    0,    5,  162,   38,   36],
       [   0,    0,    0,  151,    6, 1155,    3],
       [   0,    0,    0,  111,    1, 2367,   36],
       [   0,    0,    0,   79,  712,  247,    3],
       [   0,    0,    0,   93,   19,   16,   36],
       [   0,    0,    0,   16,    6,  173,    3],
       [   0,    0,    0,  180,

In [31]:
model(data[0][0].reshape(1,32,7))

ValueError: Layer model_1 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[   0,    0,    0,    0,   19,   60,    3],
        [   0,    0,    0,    0, 3701,   56,   36],
        [   0,    0,    0,    0,  289,   16,    3],
        [   0,    0,    0,    0, 3841,   67,    3],
        [   0,    0,    0,    0,   22,    9,    3],
        [   0,    0,    0,    0,  619, 3952,    3],
        [   0,    0,    0,    0, 1416,  169,    3],
        [   0,    0,    0,    0, 1072,   16,    3],
        [   0,    0,    0,    0,   22,  247,    3],
        [   0,    0,    0,    0, 2626,   67,    3],
        [   0,    0,    0,    0,   22,    9,    3],
        [   0,    0,    0,    0, 4118,   85,   36],
        [   0,    0,    0,   16,    8,  370,    3],
        [   0,    0,    0,    5,  162,   38,   36],
        [   0,    0,    0,  151,    6, 1155,    3],
        [   0,    0,    0,  111,    1, 2367,   36],
        [   0,    0,    0,   79,  

In [24]:
a.embedding.dtype

dtype('float64')

In [25]:
a.embedding = a.embedding.astype(np.float32, copy=False)

In [26]:
a.embedding.dtype

dtype('float32')

In [5]:
import numpy as np
a = np.random.uniform(1, 2, size=(2,3), dtype=np.float32)
a.

SyntaxError: invalid syntax (<ipython-input-5-89d6ce7bfdf7>, line 3)

In [6]:
a = TargetSequence(rest_train_fp, 32, tok, moses)


In [12]:
import numpy as np
print(a[0][0].shape)
print(a[0][1].shape)
a[0][2]
np.array([a[0][0], a[0][1], np.array(a[0][2]).reshape(32,1)])

(32, 7)
(32, 2)


ValueError: could not broadcast input array from shape (32,7) into shape (32)

In [14]:
np.array(a[0][2]).reshape(32,1).shape

(32, 1)

In [20]:
some_text[1:
          2]

['Killer Sushi!']

[["Where tanks in other Chinatown restaurants display a lurking myriad of sad-looking marine life in their murky waters, the tanks at Ping's are clear as glass with healthy-looking creatures who do not yet know that they will be part of some dim sum lover's brunch.",
  "Where tanks in other Chinatown restaurants display a lurking myriad of sad-looking marine life in their murky waters, the tanks at Ping's are clear as glass with healthy-looking creatures who do not yet know that they will be part of some dim sum lover's brunch.",
  "From the complimentary chef app of a delicate butternut squash ravioli in a delicious truffle sauce to an amazing buttery and tender langostine entree to a dessert that I can't remember because of the fabulous Cakebread Cabernet we were drinking -- the whole evening was amazing.",
  "From the complimentary chef app of a delicate butternut squash ravioli in a delicious truffle sauce to an amazing buttery and tender langostine entree to a dessert that I can't

In [12]:
a.token_index[',']

3

In [18]:
b=a._embedding_path

In [19]:
import linecache

In [38]:
linecache.getline('/home/andrew/Desktop/config.yaml', 3)

'####\n'

In [39]:
a = 'hello 50 50 50\n'
a.split()

['hello', '50', '50', '50']

In [20]:
embedding_word_line_index = {}
with b.open('r') as embedding_file:
    for index, line in enumerate(embedding_file):
        embedding_word = line.split()[0]
        embedding_word_line_index[embedding_word] = index

In [31]:
with b.open('r') as embedding_file:
    for index, line in enumerate(embedding_file):
        print(line)
        print('-----')
        break

, -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.3

In [29]:
linecache.getline(str(b.resolve()), 1)

', -0.082752 0.67204 -0.14987 -0.064983 0.056491 0.40228 0.0027747 -0.3311 -0.30691 2.0817 0.031819 0.013643 0.30265 0.0071297 -0.5819 -0.2774 -0.062254 1.1451 -0.24232 0.1235 -0.12243 0.33152 -0.006162 -0.30541 -0.13057 -0.054601 0.037083 -0.070552 0.5893 -0.30385 0.2898 -0.14653 -0.27052 0.37161 0.32031 -0.29125 0.0052483 -0.13212 -0.052736 0.087349 -0.26668 -0.16897 0.015162 -0.0083746 -0.14871 0.23413 -0.20719 -0.091386 0.40075 -0.17223 0.18145 0.37586 -0.28682 0.37289 -0.16185 0.18008 0.3032 -0.13216 0.18352 0.095759 0.094916 0.008289 0.11761 0.34046 0.03677 -0.29077 0.058303 -0.027814 0.082941 0.1862 -0.031494 0.27985 -0.074412 -0.13762 -0.21866 0.18138 0.040855 -0.113 0.24107 0.3657 -0.27525 -0.05684 0.34872 0.011884 0.14517 -0.71395 0.48497 0.14807 0.62287 0.20599 0.58379 -0.13438 0.40207 0.18311 0.28021 -0.42349 -0.25626 0.17715 -0.54095 0.16596 -0.036058 0.08499 -0.64989 0.075549 -0.28831 0.40626 -0.2802 0.094062 0.32406 0.28437 -0.26341 0.11553 0.071918 -0.47215 -0.18366 -0.

In [21]:
line_word = {line: word for word, line in embedding_word_line_index.items()}

In [30]:
line_word[0]

','

In [24]:
linecache.getline(str(b.resolve()), 1000)

'People -0.20997 -0.15582 -0.073606 -0.34189 -0.32032 0.28008 0.0033769 -0.079931 0.083 1.8469 -0.86466 -0.27001 -0.021629 -0.24913 -0.13378 0.19822 -0.57364 0.051489 -0.53949 0.23011 0.084569 0.20623 -0.18806 -0.054685 -4.9371e-05 -0.051435 0.44651 -0.022856 0.063644 -0.3956 0.2458 -0.15545 -0.45205 -0.1106 0.59292 0.24084 0.0793 -0.023429 0.084101 0.10123 0.51899 0.10656 -0.34206 0.22535 0.064097 -0.49653 -0.45743 0.05437 -0.058989 0.11791 -0.44047 0.051933 0.12841 -0.30195 0.14694 -0.084461 0.097706 -0.24829 0.068971 -0.074712 -0.032083 -0.073342 -0.20079 0.025478 0.11357 -0.24824 -0.27349 -0.10392 -0.068701 0.13297 -0.10657 0.34653 0.29558 -0.068083 0.4593 -0.4312 0.074127 0.19135 0.24887 -0.027464 -0.021497 0.27891 0.087313 -0.2332 0.059522 -0.20898 -0.99762 -0.030772 0.32078 -0.37599 -0.19092 -0.2672 0.10873 0.082608 0.01813 -0.098694 -0.084005 -0.39826 0.055193 -0.55441 -0.013784 0.35865 0.17683 -0.034674 0.52994 1.3731 0.10275 -0.38158 -0.046285 0.67172 0.14183 -0.1328 0.2049 -

In [10]:
import numpy as np
c=np.random.uniform(low=-0.05, high=0.05, size=(4554, 50))
c.shape

(4554, 50)

In [13]:
c[4554]

IndexError: index 4554 is out of bounds for axis 0 with size 4554

In [21]:
h = list(tok.word_index.items())
h[0]

('<UNK>', 1)

In [15]:
d = 'hello 50 50 50 50'
d.strip()

'hello 50 50 50 50'

In [12]:
type(c)

numpy.ndarray

In [14]:
np.random.uniform(low=-0.05, high=0.05, size=(50))

array([ 0.03436674, -0.00615127,  0.04208718,  0.00806937, -0.02588832,
       -0.04503144, -0.03547168,  0.04760892, -0.00275731, -0.03723368,
       -0.03446525,  0.02533241, -0.03804036,  0.01703367, -0.00324602,
        0.01917142, -0.04468261, -0.045914  ,  0.04559775, -0.01955871,
        0.01973594, -0.0168532 , -0.03996611, -0.01072227,  0.03983544,
       -0.00618201, -0.01242969,  0.01749668, -0.03591887,  0.03261453,
        0.03764831, -0.02657069,  0.0257145 ,  0.03268912, -0.01167673,
       -0.04949418,  0.03001373, -0.02660648, -0.01177249,  0.02424233,
        0.04264256, -0.03420635, -0.03382716,  0.01070314, -0.04667419,
       -0.00693614, -0.00454465,  0.01144933,  0.03883786,  0.01923784])

In [7]:
h = [1, 2, 3, 4, 5]
h[-2:]

[4, 5]

In [8]:
h[:-2]

[1, 2, 3]